SAM NOTES, DELETE LATER
 - need to make sure that .ipynb that student's will work out of are running in same virutal enviroment as everything else
 - embed .ipynb into quarto notebook (get book to build from those examples)
 - use Ryan Abernathey's [post](https://medium.com/pangeo/closed-platforms-vs-open-architectures-for-cloud-native-earth-system-analytics-1ad88708ebb6) to help frame introduction

## Learning Objectives

- Understand what Google Earth Engine provides and its applications
- Learn about some real-world applications of Google Earth Engine
- Learn how to get started using Google Earth Engine on your own computer
- Learn how to find and access Google Earth Engine Data

## Introduction (15-20min)

SAM NOTES, DELETE LATER 
- have Ingmar help frame utility of GEE in intro

[Google Earth Engine](https://earthengine.google.com/) is a geospatial processing platform powered by Google Cloud Platform. It contains over 30 years of satellite imagery and geospatial datasets that are continually updated and available instantly. The Earth Engine API is available in Python (and JavaScript) for anyone with an account to access and analyze data.

ADD SOME COOL IMAGERY HERE

Explore the public [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog) which includes a variety of standard Earth science raster datasets. Browse by [dataset tags](https://developers.google.com/earth-engine/datasets/tags) or by satellites ([Landsat](https://developers.google.com/earth-engine/datasets/catalog/landsat), [MODIS](https://developers.google.com/earth-engine/datasets/catalog/modis), [Sentinel](https://developers.google.com/earth-engine/datasets/catalog/sentinel)).

SAM NOTES, DELETE LATER
- typical: download data and work locally
- new way:"Moving compute to data" -- GEE is a great example of this

## Getting started with Google Earth Engine (GEE) on your own machine (40 min)

SAM NOTES, DELETE LATER

- remove `#| eval: false` once code actually runs to embed outputs
- don't really have time to actually do this in class? include instructions for those who want to try it out on their own later?
- Content borrowed from Dr. Samantha Stevenson's [guide to installling Jupyter/Google Earth Engine onyour personal laptop](https://docs.google.com/document/d/1P1c6Ef7D1OoGaTaOrPDO0m5LBN1okpGlU0OXKKzsoi0/edit).
- PREREQUISITE: need conda (see Earth Engine API installion instructions [here](https://developers.google.com/earth-engine/guides/python_install-conda)) -- haven't added these instructions below yet
- also, not working on my machine at the moment. issue with library installs? starts with `earthengine authenticate`


1. Install the Google Earth Engine API

SAM NOTES, DELETE LATER 
- MOVE MOST OF THIS TO COURSE SETUP (not using conda) BUT KEEP GEE ACTIVATION STEP HERE

- Create an environment where the Google Earth Engine API will live. This ensures that it and it's dependent packages will not cause versioning issues with your base environment (or other environments). We'll call our environment `gee_env`.


```{bash}
#| eval: false
conda create --name gee_env
```


- Activate your environment so your machine knows where to store subsequent installs.


```{bash}
#| eval: false
conda activate gee_env
```


You'll know your environment is activated successfully when `(gee_env)` appears before the prompt in your terminal window (as opposed to `(base)`, for example).

- Install the Google Earth Engine API in your `gee_env`


```{bash}
#| eval: false
conda install -c conda-forge earthengine-api
```


2. Sign up for a GEE Account

GEE is currently free for educational use. Sign up for an account at https://signup.earthengine.google.com (you'll need this to authenticate in the next step).

3. Set up GEE Authentication

In order to begin using GEE, you'll need to connect your GEE envionment (`gee_env`) to the authentication credentials associated with your Google account. This will need to be done each time you connect to GEE, but should only be done once per session. 

- On the command line, type:


```{bash}
#| eval: false
earthengine authenticate
```


This should launch a browser window where you can login with your Google account to the Google Earth Engine Authenticator. Following the prompts will generate a code, which you'll then need to copy and paste back onto the command line. This will be saved as an authentication token so you won't need to go through this process again until the next time you start a new session.

4. Install necessary packages (if you don't already have them)


```{bash}
#| eval: false
pip install ee # Earth Engine API package
pip install geemap # package for interactive maping with GEE 
pip install pandas # contains useful tools for data manipulation (may not need this)
pip install eemont # contains simplified GEE syntax
```


## Visualize global precipitation data using Google Earth Engine 

*Content for this section was adapted from Dr. Sam Stevenson's [Visualizing global precipitation using Google Earth Engine](https://github.com/samanthastevenson/EDS220_Fall2021/blob/main/Week1_ERA5Maps_GEE.ipynb) lesson, given in her [EDS 220 course](https://samanthastevenson.github.io/EDS220_site/) in Fall 2021.*

1. Import necessary packages


In [1]:
#| eval: false
import ee # MODULENOTFOUNDERROR
import geemap
import eemont
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
ee.Initialize()

2. Create an interactive basemap

The default basemap is (you guessed it) Google Maps. The following code displays an empty Google Map that you can manipulate just like you would in the typical Google Maps interface. Do this using the `Map` method from the `geemap` library. We'll also center the map at a specified latitude and longitude (here, 40N, 100E), set a zoom level, and save our map as an object called `myMap`.


In [ ]:
#| eval: false
myMap = geemap.Map(center = [40, -100], zoom = 2)
myMap

3. Load ERA5 Image Collections from GEE

* NOTE: ADC has worked with these data -- took 3 weeks to download
* EE colleciton is all you need to load and analyze imgage collection
* precursor to Ingmar's stuff

We'll be using the ERA5 daily aggregates reanalysis dataset, produced by the European Centre for Medium-Range Weather Forecasts (ECMWF), found [here](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY), which models atmospheric weather observations. We'll load the `total_precipitation` field (check out the dataset metadata on [here](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#bands)).

The `ImageCollection` method extracts a set of individual images that satisfies some criterion that you pass to GEE through the `ee` package. This is stored as an **ImageCollection** object which can be filtered and processed in various ways. We can pass the `ImageCollction` method agruments to tell GEE which data we want to retrieve. Below, we retrieve all daily ERA5 data (so we can see individual rain events).


In [ ]:
#| eval: false
weatherData = ee.ImageCollection('ECMWF/ERA5/DAILY')

4. Select an image to plot

To plot a map over our Google Maps basemap, we need an "Image" rather than an "ImageCollection." ERA5 contains many different climate variables, so we need to pick what we'd like to plot. We'll use the `.select` method to choose the parameter(s) we're interested in from our `weatherData` object.


In [ ]:
#| eval: false
precip = weatherData.select("total_precipitation")

We can look at our `precip` object using the `print` method to see that it's still an "ImageCollection" which contains daily infomration from 1979 to 2020. 


In [ ]:
#| eval: false
print(precip)

We want to filter it down to a single field for a time of interest -- let's say December 1-2, 2019. We apply the `.filter` method to our `precip` object and apply the `ee.Filter.date` method (from the `ee` package) to filter for data from our chosen date range. Always choose one more day! (Quirky GEE based behaviour!)
We apply the `.first` method to choose the first (and only image).

**deprecated**
We also apply the `.mean` method, which takes whatever precedes it and calculates the average.


In [ ]:
#| eval: false
precip_filtered = precip.filter(ee.Filter.date('2019-12-01', '2019-12-02')).first()

5. Add data to map

We can fist use the `setCenter` method to tell the map where to center itself. It takes the longitude and latitude as the first two coordinates, followed by the zoom level.


In [ ]:
#| eval: false
myMap.setCenter(-152.505706, 59.432367, 2) # Cook Inlet, Alaska (WE CAN CHANGE THIS LOCATION)

Next, set a color palette to use when plotting the data layer. The following is a palette specified for precipitation in the GEE description page for ERA5. GEE has lots of color tables like this that you can look up. 


In [ ]:
#| eval: false
palette = ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
precip_viz_params = {
    'min':0,
    'max':0.1,
    'palette': palette
}

Finally, plot our filtered data, `precip_filtered` on top of our basemap using the `.addLayer` method. We'll aslo pass it our visualization parameters (colors and ranges stored in `precip_palette`, the name of the data field `total precipitation`, and opacity so that we can see the basemap underneath)


In [ ]:
#| eval: false
myMap.addLayer(precip_filtered, precip_viz_params, 'total precipitation 2019-12-01', opacity = 0.3)
myMap

#### Annually aggregated data 
* calculate sum of precipitation
* scale to mm

In [ ]:
precip_annual = precip.filter(ee.Filter.date('2019-01-01', '2020-01-01')).sum().multiply(1000)

myMap.addLayer(precip_annual, dict(min=0, max=5000, palette=palette), 'Annual Precipitation')
myMap

## INGMAR'S DEMONSTRATION HERE?(30-40 min)
* optical remote sensing
1. load and visualize image
2. calculate and visualize indices
3. Work with ImageCollections
4. Extract time-series
5. clustering / unsupervized classification

Here we want show show how to visualize and analyze fire dynamics in the Arctic

### Loading and visualizing Data

#### Landscape changes in Arctic permafrost Regions
* Slope Processes (thaw slumps, landslides)
* Wildfires
* Hydrological Changes
* Coastal Dynamics

#### Area of Interest - Batagay

<p float="left">
    <img src="https://www.science.org/do/10.1126/science.abe0581/full/main_batagai_1280p2.jpg" style="width:400px;height:300px;">
    <img src="../images/MODIS_snapshot-2021-08-13T00_00_00Z.jpg" style="width:400px;height:300px;">
</p>

Source: https://www.science.org/do/10.1126/science.abe0581/full/main_batagai_1280p2.jpg

Source: https://worldview.earthdata.nasa.gov/

#### Location : Batagay
* define aoi
* check out basemaps (e.g. OpenTopMap)

In [3]:
# setup site and show in Map
xy = [134.6, 67.66]
poi = ee.Geometry.Point(xy) # Point
bbox = ee.Geometry.BBox(134.45, 67.7, 134.8, 67.5970) # Polygon

In [ ]:
Map = geemap.Map()
Map.addLayer(bbox, {}, 'AOI')
Map.centerObject(poi, zoom=3)
Map

#### Visualize Satellite Data
**1. Image Visualizations**
* RGB: Red - Green - Blue
* Color-Infrared: Nir - Red - Green

**2. Multispectral Indices**
* NDVI: Normalized Difference Vegetation Index
    * Good for vegetation
* NBR: Normalized Burn Ratio
    * Good for burned areas

**3. Draw point and extract to EE Feature**

Instantiate Map

In [ ]:
Map = geemap.Map()

Load Map

In [ ]:
# load image
S2_Image = (ee.Image('COPERNICUS/S2_SR/20210805T030551_20210805T030545_T53WMR')
            .scaleAndOffset()
            .spectralIndices(['NDVI', 'NBR']) # Calculate multispectral indices
           )
# Show

Map.addLayer(S2_Image, dict(min=0.0, max=0.1, gamma=1.5, bands=['B4', 'B3', 'B2']), 'RGB')
Map.addLayer(S2_Image, dict(min=0.0, max=0.3, gamma=1.5, bands=['B8', 'B4', 'B3']), 'Color-Infrared', False)

RdYlGn = ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']
Map.addLayer(S2_Image, dict(min=0.0, max=0.8, bands=['NDVI'], palette=RdYlGn), 'NDVI', False)
Map.addLayer(S2_Image, dict(min=0.0, max=0.8, bands=['NBR'], palette=RdYlGn), 'NBR', False)

Map.centerObject(S2_Image, zoom=8)

In [ ]:
Map

#### Select Point in Map above and pull location as ee.Feature

In [ ]:
point_fire = Map.draw_last_feature

backup option

In [ ]:
#point_fire = ee.Geometry.Point([134.6339, 68.0397])

#### Load more than image + analyze them
* you can do the same analysis with a ImageCollections (stacks of images of the same type)
* Load full S2 series
* filter
    * Location
    * Metadata - e.g. cloud cover
    * Dates 
        * Start + End
        * Specific months: June-September

In [ ]:
# Image Collection
ds = 'COPERNICUS/S2_SR'

S2 = (ee.ImageCollection(ds)
      ### Filters ###
      .filterBounds(poi) # only images, which intersect our point/region
      .filter(ee.Filter.calendarRange(6,9,'month')) # only June through September, Summer only
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) # only images with less that 10 % clouds
      #.filterDate('2022-07-01','2022-12-31')
      ### PreProcessing ###
      .scaleAndOffset()
      .spectralIndices(['NDVI', 'NBR'])
      .maskClouds()
      .sort('CLOUDY_PIXEL_PERCENTAGE')
     )#.first()
# calculate indices (e.g. NDVI, NBR)
print(f'Number of images {S2.size().getInfo()}') # Print number of images

### Make time-series 
Example from https://eemont.readthedocs.io/en/0.2.0/guide/timeSeries.html

Plot values

### Now let's try a longer time-series
* S2_SR only has 4 years
* e.g. Landsat / MODIS

In [4]:
def load_and_prepare_Landsat(collection_name):
    collection = (ee.ImageCollection(collection_name)
      ### Filters ###
      .filterBounds(bbox) # only images, which intersect our point/region
      .filter(ee.Filter.calendarRange(7,8,'month')) # only June through September, Summer only
      .filter(ee.Filter.lt('CLOUD_COVER_LAND', 10)) # only images with less that 10 % clouds
      #.filterDate('2022-07-01','2022-12-31')
      ### PreProcessing ###
      .scaleAndOffset()
      .spectralIndices(['NDVI', 'NBR'])
      .maskClouds()
     )
    return collection

In [5]:
L9 = load_and_prepare_Landsat("LANDSAT/LC09/C02/T1_L2")
L8 = load_and_prepare_Landsat("LANDSAT/LC08/C02/T1_L2")
L7 = load_and_prepare_Landsat("LANDSAT/LE07/C02/T1_L2")
L5 = load_and_prepare_Landsat("LANDSAT/LT05/C02/T1_L2")
LS = L9.merge(L8).merge(L7).merge(L5)

In [6]:
ts = LS.getTimeSeriesByRegion(reducer = ee.Reducer.mean(),
                              geometry = point_fire,
                              bands = ['NBR','NDVI'],
                              scale = 10)

NameError: name 'point_fire' is not defined

In [ ]:
tsPandas = geemap.ee_to_pandas(ts)

In [ ]:
tsPandas

#### Some more cleaning necessary 

In [ ]:
tsPandas[tsPandas == -9999] = np.nan
tsPandas['date'] = pd.to_datetime(tsPandas['date'],infer_datetime_format = True)

tsPandas = tsPandas.sort_values(by='date').dropna(axis=0).reset_index()

In [ ]:
tsPandas

### Visualize output 
* example with Plotly
   * or use package/library of your choice: matplotlib, seaborn, ...

In [ ]:
px.scatter(tsPandas, x='date', y=['NDVI', 'NBR'])

In [ ]:
px.line(tsPandas, x='date', y=['NDVI', 'NBR'])

### annual composite 

In [ ]:
LS_2020 = LS.filterDate('2020-01-01', '2020-12-31')
LS_2019 = LS.filterDate('2019-01-01', '2019-12-31')

In [ ]:
LS_2020_reduced = LS_2020.reduce(ee.Reducer.median())
LS_2019_reduced = LS_2019.reduce(ee.Reducer.median())

In [ ]:
Map = geemap.Map()
Map.addLayer(LS_2020_reduced,dict(min=-1, max=1, bands=['NDVI_median']), 'Median NDVI 2020')
Map.addLayer(LS_2019_reduced,dict(min=-1, max=1, bands=['NDVI_median']), 'Median NDVI 2019')
#Map.centerObject(LS_2020_reduced, 9)
Map

### Calculate Trend 

In [7]:
def make_dateband(image):
  factor = ee.Number(864000000000) # seconds to years
  time = ee.Number(image.get("system:time_start"))
  time2 = time.toDouble().divide(factor)
  date_image = ee.Image.constant(time2).toFloat().select([0], ['Date'])
  #TODO: rename to 'Date'
  return image.addBands(date_image)

In [8]:
LS = LS.map(make_dateband)

In [9]:
index = 'NDVI'
trend_image = ee.Image()
trend = ee.ImageCollection(LS.select(['Date', index])).reduce(ee.Reducer.linearFit().unweighted())
trend_image = trend_image.addBands(trend)

In [10]:
palette = geemap.get_palette_colors(cmap_name='RdYlGn')

In [13]:
Map = geemap.Map()
Map.addLayer(trend_image.clip(bbox), dict(min=-0.2, max=0.2, palette=palette, bands=['scale']))
#Map.centerObject(trend_image, 8)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

Add some nice temporal data
Tasseled Cap Trends: 2001-2020

In [15]:
TCVIS = ee.ImageCollection('users/ingmarnitze/TCTrend_SR_2001-2020_TCVIS')
Map.addLayer(TCVIS, {}, 'TCVIS')

### Clustering 
* Group individual image into several "similar" objects
* taken from: https://geemap.org/notebooks/31_unsupervised_classification/#make-training-dataset

In [ ]:
image = S2_Image

In [ ]:
# Make the training dataset.
training = image.sample(
    **{
        #     'region': region,
        'scale': 10,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

In [ ]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')


In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)

In [ ]:
Map

## Conclusion/Summary 

* lessons learned
* utilities
* etc.

## Other Resources

### Datasets
- [GEE Code Editor](https://code.earthengine.google.com/) is a web-based IDE for using GEE (JavaScript)
- [GEE Datasets](https://developers.google.com/earth-engine/datasets) is the main resource for "official" GEE Datasets
- [GEE Community Datasets](https://samapriya.github.io/awesome-gee-community-datasets/) is the main resource for "community" GEE Datasets

### Tutorials

* **Qiusheng Wu's Youtube Channel !!!** https://www.youtube.com/c/Qiushengwu

* **Documentation and Tutorials** https://geemap.org/

* **EEmont** Documentation: https://eemont.readthedocs.io/en/latest/

### Stackexchange
* https://gis.stackexchange.com/questions/tagged/google-earth-engine GEE Stackexchange